In [27]:
import torch
from transformers import ViTModel, ViTImageProcessor, AutoModel, ViTImageProcessor
from PIL import Image
import os
from tqdm import tqdm
import pandas as pd
import numpy

In [28]:
def load_models(modelo):

    if modelo == 'ViT_huge':
        vit_huge_model = ViTModel.from_pretrained('google/vit-huge-patch14-224-in21k')
        vit_huge_feature_extractor = ViTImageProcessor.from_pretrained('google/vit-huge-patch14-224-in21k')
        vit_huge_model.eval()

        return vit_huge_model, vit_huge_feature_extractor

    if modelo == 'ViT_large':
        vit_large_model = ViTModel.from_pretrained('google/vit-large-patch16-224-in21k')
        vit_large_feature_extractor = ViTImageProcessor.from_pretrained('google/vit-large-patch16-224-in21k')
        vit_large_model.eval()

        return vit_large_model, vit_large_feature_extractor
    
    if modelo == 'ViT_base':
        vit_base_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        vit_base_feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
        vit_base_model.eval()
    
        return vit_base_model, vit_base_feature_extractor

    if modelo == 'ViT_small':
        vit_small_model = AutoModel.from_pretrained('WinKawaks/vit-small-patch16-224')
        vit_small_feature_extractor = ViTImageProcessor.from_pretrained('WinKawaks/vit-small-patch16-224')
        vit_small_model.eval()

        return vit_small_model, vit_small_feature_extractor

    return 


In [33]:
def feature_extraction(image_path, modelo, feature_extractor):
    image = Image.open(image_path).convert('RGB')
    inputs = feature_extractor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = modelo(**inputs)
    
    last_hidden_state = outputs.last_hidden_state

    features = last_hidden_state.mean(dim=1).squeeze().numpy()
    
    return features

In [40]:
def features_to_df(folder_path, modelo, feature_extractor):
    data = []
    
    for subfolder in tqdm(os.listdir(folder_path)):
        subfolder_path = os.path.join(folder_path, subfolder)
    
        if os.path.isdir(subfolder_path):
            for image_file in os.listdir(subfolder_path):
                image_path = os.path.join(subfolder_path, image_file)
                
                if image_file.lower().endswith(('png', 'jpg', 'jpeg')):
                    features = feature_extraction(image_path, modelo, feature_extractor)
                        
                    data.append([image_path, *features])

    columns = ['image_path'] + [f'feature_{i}' for i in range(len(features))]
    df = pd.DataFrame(data, columns=columns)
    return df

    

In [45]:
def save_dataframe_to_csv(df, caminho, modelo):
    path = os.path.join(caminho)
    df.to_csv(path+'/'+f'Features_{modelo}'+'.csv', index=False)
    print(f"Arquivos do {modelo} salvo em: {path}")

In [ ]:
modelo
caminho_absoluto_fonte
caminho_absoluto_salvar

In [41]:
model, fe = load_models(modelo)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
df = features_to_df(caminho_absoluto, model, fe)

  0%|                                                     | 0/5 [00:06<?, ?it/s]

KeyboardInterrupt



In [49]:
save_dataframe_to_csv(df, caminho_absoluto_salvar, nome_arquivo_salvar, modelo)

Arquivos do ViT_small salvo em: ./features_ViT_small
